In [1]:
import sys
sys.path.insert(0, '../')
from qhdopt import QHD
import numpy as np
import mpmath

/Users/samuelkushnir/miniconda3/envs/QHDOPT-dev/lib/python3.9/site-packages/qutip/__init__.py:65: UserWarning: The new version of Cython, (>= 3.0.0) is not supported.
  warnings.warn(


In [14]:
Q = np.array([[0.8931, 0.8673],
     [0.8673, 0.9079]])
b = np.array([0.8483, 0.8762])

In [2]:
def to_numpy(v):
    return np.array(v.tolist(), dtype=np.float64)

In [3]:
def get_decimal_part(float_str):
    # Split the string on the decimal point
    parts = float_str.split('.')
    # Check if there is a decimal part and return it
    if len(parts) > 1:
        return parts[1]
    else:
        # If there's no decimal part, return an empty string
        return ''

In [4]:
def get_first_nonzero_decimal_place(n):
    n_str = get_decimal_part(str(n))
    for i in range(len(n_str)):
        if n_str[i] != "0":
            return i + 1
    return len(n_str)

In [19]:
def run_qhd(matrix, vector):
    first_nonzero_decimal_place = min([get_first_nonzero_decimal_place(v_i) for v_i in vector])
    if first_nonzero_decimal_place <= 1:
        offset = 0
    else:
        offset = first_nonzero_decimal_place - 1
    vector = to_numpy(-1*(vector * 10 ** offset)).squeeze()
    model = QHD.QP(matrix, vector)
    model.dwave_setup(20, api_key_from_file='/Users/samuelkushnir/Documents/dwave_api_key.txt', embedding_scheme="hamming")
    model.optimize(verbose=0, fine_tune=False)
    return model.info["coarse_minimizer"], offset


def IR_QHD(matrix, vector, IRprecision):
  ### Scaled Iterative Refinement for solving a linear system
  ### matrix: coefficient matrix of the linear system
  ### vector: right-hand side vector of the linear system
    nabla             = 1                             # Scaling factor
    rho               = 2                             # Incremental scaling
    d                 = len(matrix)                   # Dimension
    iteration         = 0
    x                 = mpmath.matrix(np.zeros(d))                 # Solution
    r                 = vector                        # Residual
    # con               = np.linalg.cond(matrix)             # Condition number
    res=[]
    while (np.linalg.norm(r)>IRprecision):
        c, offset = run_qhd(to_numpy(matrix), nabla*r)
        c = mpmath.matrix(c)
        c = c * 10 ** -offset
        x = x + (1/nabla)*c                           # Updating solution
        r = vector - matrix * x               # Calculating resisdual
        # nabla = min(rho*nabla,1/(np.linalg.norm(r)))  # Updating scaling factor
        res.append(np.linalg.norm(r))
        iteration=iteration+1
    return res, c

# Set the desired precision (number of decimal places)
mpmath.mp.dps = 50  # for example, 50 decimal places

# Create the matrix A using numpy
A = np.random.rand(2, 2)

# Convert A to mpmath matrix and make it symmetric to ensure positive-definiteness
A_mpmath = mpmath.matrix(A)
A_mpmath = A_mpmath * A_mpmath.transpose()

# High precision x vector
x_mpmath = mpmath.matrix([0.34987313872918378423, 0.534563452331234876327])

# Calculate b using mpmath's dot product
b_mpmath = A_mpmath * x_mpmath

# Convert results back to numpy arrays for any further processing
A_np = np.array(A_mpmath.tolist(), dtype=np.float64)
b_np = np.array(b_mpmath.tolist(), dtype=np.float64)


print(IR_QHD(A_mpmath, b_mpmath, 1e-4))

Received Task from D-Wave:
2024-02-02 17:22:39
Backend QPU Time: 0.03131477
Overhead Time: 6.132676442844849

Received Task from D-Wave:
2024-02-02 17:25:58
Backend QPU Time: 0.027887569999999997
Overhead Time: 7.128057492637329



KeyboardInterrupt: 